<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Отворен курс за машинно обучение

Автор: [Юрий Кашницки](https://yorko.github.io). Превод и редакция от [Christina Butsko](https://www.linkedin.com/in/christinabutsko/), [Nerses Bagiyan](https://www.linkedin.com/in/nersesbagiyan/), [Yulia Klimushina] (https://www.linkedin.com/in/yuliya-klimushina-7168a9139) и [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/). Този материал е предмет на правилата и условията на лиценза [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Безплатното използване е разрешено за всякакви нетърговски цели.

# <center>Тема 4. Линейна класификация и регресия
## <center> Част 4. Къде логистичната регресия е добра и къде не
    
            
## Описание на статията
1. [Анализ на рецензии за филми в IMDB](#1.-Анализ-на-ревюта-на-IMDB-филми)
2. [Просто броене на думи](#2.-Просто броене-на-думи)
3. [XOR-Проблем](#3.-XOR-Проблем)
4. [Демо задание](#4.-Демо-задаване)
5. [Полезни ресурси](#5.-Полезни-ресурси)

## 1. Анализ на рецензии за филми в IMDB

Сега за малко практика! Искаме да разрешим проблема с двоичната класификация на рецензиите на филми в IMDB. Имаме тренировъчен комплект с отбелязани отзиви, 12500 отзива, маркирани като добри, още 12500 лоши. Тук не е лесно да започнем с машинното обучение веднага, защото нямаме матрицата $X$; трябва да го подготвим. Ще използваме прост подход: модел на торба с думи. Характеристиките на рецензията ще бъдат представени чрез индикатори за наличието на всяка дума от целия корпус в тази рецензия. Корпусът е набор от всички потребителски рецензии. Идеята е илюстрирана със снимка
<img src="../../img/bag_of_words.svg" width=80% />

In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

**За да започнете, автоматично изтегляме набора от данни от [тук](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) и го деархивираме заедно с останалите набори от данни в папка с данни. Наборът от данни е описан накратко [тук](http://ai.stanford.edu/~amaas/data/sentiment/). Има 12,5 хиляди добри и лоши отзиви в тестовите и тренировъчни комплекти.**

In [2]:
import tarfile
from io import BytesIO

import requests

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"


def load_imdb_dataset(extract_path="../../data", overwrite=False):
    # check if existed already
    if (
        os.path.isfile(os.path.join(extract_path, "aclImdb", "README"))
        and not overwrite
    ):
        print("IMDB dataset is already in place.")
        return

    print("Downloading the dataset from:  ", url)
    response = requests.get(url)

    tar = tarfile.open(mode="r:gz", fileobj=BytesIO(response.content))

    data = tar.extractall(extract_path)


load_imdb_dataset()

In [1]:
# change if you have it in alternative location
PATH_TO_IMDB = "../../data/aclImdb"

reviews_train = load_files(
    os.path.join(PATH_TO_IMDB, "train"), categories=["pos", "neg"]
)
text_train, y_train = reviews_train.data, reviews_train.target

reviews_test = load_files(os.path.join(PATH_TO_IMDB, "test"), categories=["pos", "neg"])
text_test, y_test = reviews_test.data, reviews_test.target

NameError: name 'load_files' is not defined

In [3]:
# # Алтернативно, заредете данни от предварително избрани обекти.
# import pickle
# с open('../../data/imdb_text_train.pkl', 'rb') as f:
# text_train = pickle.load(f)
# с open('../../data/imdb_text_test.pkl', 'rb') as f:
# text_test = pickle.load(f)
# с open('../../data/imdb_target_train.pkl', 'rb') as f:
# y_train = pickle.load(f)
# с open('../../data/imdb_target_test.pkl', 'rb') as f:
# y_test = pickle.load(f)

In [5]:
print("Number of documents in training data: %d" % len(text_train))
print(np.bincount(y_train))
print("Number of documents in test data: %d" % len(text_test))
print(np.bincount(y_test))

Number of documents in training data: 25000
[12500 12500]
Number of documents in test data: 25000
[12500 12500]


**Ето няколко примера за отзиви.**

In [6]:
print(text_train[1])

b'Words can\'t describe how bad this movie is. I can\'t explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many clich\xc3\xa9s, mistakes (and all other negative things you can imagine) here that will just make you cry. To start with the technical first, there are a LOT of mistakes regarding the airplane. I won\'t list them here, but just mention the coloring of the plane. They didn\'t even manage to show an airliner in the colors of a fictional airline, but instead used a 747 painted in the original Boeing livery. Very bad. The plot is stupid and has been done many times before, only much, much better. There are so many ridiculous moments here that i lost count of it really early. Also, I was on the bad guys\' side all the time in the movie, because the good guys were so stupid. "Executive Decision" should without a doubt be you\'re choice over this one, even the "Turbulenc

In [7]:
y_train[1]  # bad review

0

In [8]:
text_train[2]

b'Everyone plays their part pretty well in this "little nice movie". Belushi gets the chance to live part of his life differently, but ends up realizing that what he had was going to be just as good or maybe even better. The movie shows us that we ought to take advantage of the opportunities we have, not the ones we do not or cannot have. If U can get this movie on video for around $10, it\xc2\xb4d be an investment!'

In [9]:
y_train[2]  # good review

1

In [10]:
# import pickle
# with open('../../data/imdb_text_train.pkl', 'wb') as f:
#     pickle.dump(text_train, f)
# with open('../../data/imdb_text_test.pkl', 'wb') as f:
#     pickle.dump(text_test, f)
# with open('../../data/imdb_target_train.pkl', 'wb') as f:
#     pickle.dump(y_train, f)
# with open('../../data/imdb_target_test.pkl', 'wb') as f:
#     pickle.dump(y_test, f)

## 2. Просто броене на думи

**Първо ще създадем речник на всички думи с помощта на CountVectorizer**

In [11]:
cv = CountVectorizer()
cv.fit(text_train)

len(cv.vocabulary_)

74849

**Ако погледнете примерите за „думи“ (да ги наречем токени), можете да видите, че сме пропуснали много от важните стъпки в обработката на текст (самата автоматична обработка на текст може да бъде напълно отделна поредица от статии).* *

In [ ]:
print(cv.get_feature_names()[:50])
print(cv.get_feature_names()[50000:50050])

**Второ, ние кодираме изреченията от текстовете на учебния набор с индексите на входящите думи. Ще използваме разредения формат.**

In [ ]:
X_train = cv.transform(text_train)
X_train

**Нека да видим как работи нашата трансформация**

In [ ]:
print(text_train[19726])

In [ ]:
X_train[19726].nonzero()[1]

In [ ]:
X_train[19726].nonzero()

**Трето, ще приложим същите операции към тестовия набор**

In [ ]:
X_test = cv.transform(text_test)

**Следващата стъпка е да обучите логистична регресия.**

In [ ]:
%%time
logit = LogisticRegression(solver="lbfgs", n_jobs=-1, random_state=7)
logit.fit(X_train, y_train)

**Нека да разгледаме точността както на комплектите за обучение, така и на тестовите.**

In [ ]:
round(logit.score(X_train, y_train), 3), round(logit.score(X_test, y_test), 3),

**Коефициентите на модела могат да бъдат красиво показани.**

In [ ]:
def visualize_coefficients(classifier, feature_names, n_top_features=25):
    # get coefficients with large absolute values
    coef = classifier.coef_.ravel()
    positive_coefficients = np.argsort(coef)[-n_top_features:]
    negative_coefficients = np.argsort(coef)[:n_top_features]
    interesting_coefficients = np.hstack([negative_coefficients, positive_coefficients])
    # plot them
    plt.figure(figsize=(15, 5))
    colors = ["red" if c < 0 else "blue" for c in coef[interesting_coefficients]]
    plt.bar(np.arange(2 * n_top_features), coef[interesting_coefficients], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(
        np.arange(1, 1 + 2 * n_top_features),
        feature_names[interesting_coefficients],
        rotation=60,
        ha="right",
    );

In [ ]:
def plot_grid_scores(grid, param_name):
    plt.plot(
        grid.param_grid[param_name],
        grid.cv_results_["mean_train_score"],
        color="green",
        label="train",
    )
    plt.plot(
        grid.param_grid[param_name],
        grid.cv_results_["mean_test_score"],
        color="red",
        label="test",
    )
    plt.legend();

In [ ]:
visualize_coefficients(logit, cv.get_feature_names())

**За да направим нашия модел по-добър, можем да оптимизираме коефициента на регулация за „Логистичната регресия“. Ще използваме `sklearn.pipeline`, тъй като `CountVectorizer` трябва да се прилага само към данните за обучение (за да не "надникваме" в тестовия набор и да не броим честотите на думите там). В този случай `pipeline` определя правилната последователност от действия: приложете `CountVectorizer`, след това обучете `Logistic Regression`.**

In [ ]:
%%time
from sklearn.pipeline import make_pipeline

text_pipe_logit = make_pipeline(
    CountVectorizer(),
    # for some reason n_jobs > 1 won't work
    # with GridSearchCV's n_jobs > 1
    LogisticRegression(solver="lbfgs", n_jobs=1, random_state=7),
)

text_pipe_logit.fit(text_train, y_train)
print(text_pipe_logit.score(text_test, y_test))

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid_logit = {"logisticregression__C": np.logspace(-5, 0, 6)}
grid_logit = GridSearchCV(
    text_pipe_logit, param_grid_logit, return_train_score=True, cv=3, n_jobs=-1
)

grid_logit.fit(text_train, y_train)

**Нека отпечатаме най-добрия $C$ и cv-резултат, използвайки този хиперпараметър:**

In [ ]:
grid_logit.best_params_, grid_logit.best_score_

In [ ]:
plot_grid_scores(grid_logit, "logisticregression__C")

За набора за валидиране:

In [ ]:
grid_logit.score(text_test, y_test)

** Сега нека направим същото с произволна гора. Виждаме, че с логистична регресия постигаме по-добра точност с по-малко усилия.**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=17)

In [ ]:
%%time
forest.fit(X_train, y_train)

In [ ]:
round(forest.score(X_test, y_test), 3)

### 3. XOR-проблем
Нека сега разгледаме пример, при който линейните модели са по-лоши.

Методите за линейна класификация все още дефинират много проста разделителна повърхност - хиперравнина. Най-известният пример за играчка, където класовете не могат да бъдат разделени от хиперравнина (или линия) без грешки, е „проблемът XOR“.

XOR е "изключително ИЛИ", булева функция със следната таблица на истината:



<img src='../../img/XOR_table.gif'>

XOR е името, дадено на прост проблем с двоична класификация, в който класовете са представени като диагонално разширени пресичащи се облаци от точки.

In [ ]:
# creating dataset
rng = np.random.RandomState(0)
X = rng.randn(200, 2)
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired);

Очевидно не може да се начертае една права линия, за да се раздели един клас от друг без грешки. Следователно логистичната регресия се справя зле с тази задача.

In [ ]:
def plot_boundary(clf, X, y, plot_title):
    xx, yy = np.meshgrid(np.linspace(-3, 3, 50), np.linspace(-3, 3, 50))
    clf.fit(X, y)
    # plot the decision function for each datapoint on the grid
    Z = clf.predict_proba(np.vstack((xx.ravel(), yy.ravel())).T)[:, 1]
    Z = Z.reshape(xx.shape)

    image = plt.imshow(
        Z,
        interpolation="nearest",
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        aspect="auto",
        origin="lower",
        cmap=plt.cm.PuOr_r,
    )
    contours = plt.contour(xx, yy, Z, levels=[0], linewidths=2, linetypes="--")
    plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(r"$x_1$")
    plt.ylabel(r"$x_2$")
    plt.axis([-3, 3, -3, 3])
    plt.colorbar(image)
    plt.title(plot_title, fontsize=12);

In [ ]:
plot_boundary(
    LogisticRegression(solver="lbfgs"), X, y, "Logistic Regression, XOR problem"
)

Но ако трябва да се дадат полиномни характеристики като вход (тук, до 2 степени), тогава проблемът е решен.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
logit_pipe = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=2)),
        ("logit", LogisticRegression(solver="lbfgs")),
    ]
)

In [ ]:
plot_boundary(logit_pipe, X, y, "Logistic Regression + quadratic features. XOR problem")

Тук логистичната регресия все още е създала хиперравнина, но в 6-измерно пространствено пространство $1, x_1, x_2, x_1^2, x_1x_2$ и $x_2^2$. Когато проектираме към оригиналното пространствено пространство, $x_1, x_2$, границата е нелинейна.

На практика полиномните характеристики наистина помагат, но е неефективно от изчислителна гледна точка да се създават изрично. SVM с трика на ядрото работи много по-бързо. При този подход се изчислява само разстоянието между обектите (дефинирано от функцията на ядрото) във високомерно пространство и няма нужда да се създава комбинаторно голям брой характеристики. 

## 4. Демо задание
За да практикувате с линейни модели, можете да завършите [това задание](https://www.kaggle.com/kashnitsky/a4-demo-sarcasm-detection-with-logit), където ще изградите модел за откриване на сарказъм. Заданието е само за упражняване и върви с [решение](https://www.kaggle.com/kashnitsky/a4-demo-sarcasm-detection-with-logit-solution).

## 5. Полезни ресурси
- Средна ["история"](https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-4-linear-classification-and-regression-44a41b9b5220) въз основа на този бележник
- Основен курс [сайт](https://mlcourse.ai), [репо за курс](https://github.com/Yorko/mlcourse.ai) и YouTube [канал](https://www.youtube. com/watch?v=QKTuw4PNOsU&list=PLVlY_7IJCMJeRfZ68eVfEcu-UcN9BbwiX)
- Материали за курса като [Набор от данни на Kaggle](https://www.kaggle.com/kashnitsky/mlcourse)
- Ако четете руски: [статия](https://habrahabr.ru/company/ods/blog/323890/) на Habr.com със ~ същия материал. И [лекция](https://youtu.be/oTXGQ-_oqvI) в YouTube
- В книгата ["Deep Learning"] (http://www.deeplearningbook.org) (I. Goodfellow, Y. Bengio и A. Courville) е даден хубав и кратък преглед на линейните модели.
- Линейните модели са разгледани практически във всяка книга за ML. Препоръчваме „Разпознаване на модели и машинно обучение“ (C. Bishop) и „Машинно обучение: вероятностна гледна точка“ (K. Murphy).
- Ако предпочитате задълбочен преглед на линейния модел от гледна точка на статистик, тогава вижте "Елементите на статистическото обучение" (T. Hastie, R. Tibshirani и J. Friedman).
- Книгата "Машинно обучение в действие" (P. Harrington) ще ви преведе през имплементации на класически ML алгоритми в чист Python.
- [Scikit-learn](http://scikit-learn.org/stable/documentation.html) библиотека. Тези момчета работят усилено върху писането на наистина ясна документация.
- Scipy 2017 [урок за scikit-learn](https://github.com/amueller/scipy-2017-sklearn) от Алекс Грамфорт и Андреас Мюлер.
- Още един [ML курс](https://github.com/diefimov/MTH594_MachineLearning) с много добри материали.
- [Имплементации](https://github.com/rushter/MLAlgorithms) на много ML алгоритми. Търсете линейна регресия и логистична регресия.